In [1]:
from datetime import date, datetime, timedelta, time
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly

In [2]:
pu = pd.read_csv("../taxi_data/pu_predictions.csv", index_col=[0], parse_dates=["dt"])
do = pd.read_csv("../taxi_data/do_predictions.csv", index_col=[0], parse_dates=["dt"])

In [3]:
def my_plot(date_str, zones, data):
    #load zone details
    loc_zones = pd.read_csv('../taxi_data/taxiZoneLookup.csv')
    time = ["{}:00-{}:00".format(f"{i:02d}", f"{i+1:02d}") for i in range(24)]
    x_left, x_right, headers = list(), list(), list()
    for zone in zones:
        pu = data[0][data[0].PULocationID==zone].set_index("dt")[date_str]
        do = data[1][data[1].location==zone].set_index("dt")[date_str]

        new_index = [pu.index[0]]
        while new_index[-1] < pu.index[-1]:
            new_index.append(new_index[-1] + timedelta(hours=1))

        do = do.loc[~do.index.duplicated(keep='first')].reindex(new_index).fillna(method='ffill').reset_index()
        pu = pu.loc[~pu.index.duplicated(keep='first')].reindex(new_index).fillna(method='ffill').reset_index()

        x_left.append(pu.predicted_trips.tolist())
        x_right.append(do.predicted_trips.tolist())
        
        headers.append("{}, {} ({})".format(loc_zones.loc[zone-1, "Zone"], 
                                            loc_zones.loc[zone-1, "Borough"], 
                                            str(zone)
                                           ))
    
    
    # get subplot size
    if len(zones) % 3 == 0:
        rows, cols = len(zones) // 3, 3
    elif len(zones) < 3:
        rows, cols = 1, len(zones)
    else:
        rows, cols = len(zones) // 3 + 1, 3
    
    fig = plotly.tools.make_subplots(rows = rows, cols = cols, subplot_titles = headers)

    trace_rm1 = go.Bar(
        x=[i for i in reversed(x_left[0])], y=[i for i in reversed(time)], base=[-i for i in reversed(x_left[0])],
        orientation='h', marker_color='cornflowerblue', name="Pickups", showlegend=True)

    trace_rm2 = go.Bar(
        x=[i for i in reversed(x_right[0])], y=[i for i in reversed(time)],
        orientation='h', marker_color='lightblue', name="Dropoffs", showlegend=True)
    
    j = 0
    for row in range(1,rows+1):
        for col in range(1,cols+1):
            if j < len(zones):
                fig.append_trace(go.Bar(
                    x=[i for i in reversed(x_left[j])],
                    y=[i for i in reversed(time)],
                    base=[-i for i in reversed(x_left[j])],
                    orientation='h',
                    marker_color='cornflowerblue',
                    text=[i for i in reversed(x_left[j])],
                    name="Pickups",
                    showlegend=False
                ), row, col)
                fig.append_trace(go.Bar(
                    x=[i for i in reversed(x_right[j])],
                    y=[i for i in reversed(time)],
                    orientation='h',
                    marker_color='lightblue',
                    text=[i for i in reversed(x_right[j])],
                    name="Dropoffs",
                    showlegend=False
                ), row, col)
                fig['layout']['xaxis' + str(j+1)].update(title='', range=[-1500,1500])
                j += 1
    fig.update_xaxes(showticklabels=False) 
    fig.update_yaxes(showticklabels=False)   
    fig.update_layout(
            title_x=0.5,
            title=date_str,
            xaxis = dict(range=(-1500, 1500)),
            plot_bgcolor='white',
            barmode='stack',
            showlegend=True,
            legend=dict(x=0,y=1.0)
        )
    
    plotly.offline.plot(fig)

In [179]:
my_plot("2019-11-2", [236, 43, 79, 161, 100, 148], [pu, do])

In [177]:
while True:
    new_date = str(input("Choose date (YYYY-MM-DD): "))
    new_zones = str(input("Choose zones (up to 9): "))

    new_zones = [int(i) for i in new_zones.split(",")]
    my_plot(new_date, new_zones, [pu, do])

Choose date (YYYY-MM-DD): 2019-10-10
Choose zones (up to 9): 43,100,79,181


KeyboardInterrupt: 